In [ ]:
#import libraries
import cmath
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.environ import *

#load data
#Network data
xls = pd.ExcelFile('data\case6.xlsx')
#Feasible samples
xls_load = pd.ExcelFile('data/samples/6bus.xlsx')
#BusLoad data (sampling from a distribution)
xls_load_sample = pd.ExcelFile('data/samples/load.xlsx')

load_data = pd.read_excel(xls_load,'valid_samples (100)')
load_data_sample = pd.read_excel(xls_load_sample,'6bus_load')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')

#Y matrix
base = 100
bus_len = bus.shape[0]
gen_len = len(gen['bus'].ravel())

Ybus = np.matrix(np.zeros([bus_len,bus_len]),dtype=complex)
shunt = np.zeros(bus_len, dtype=complex)
x = [pd.DataFrame() for i in range(bus_len)]
for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        Ybus[fro.loc[j][0]-1,i-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        shunt[j] = complex(0,sum(x[i]['b'])/2)

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1) + complex(0,sum(branch.loc[(branch['fbus']==i+1) | (branch['tbus']==i+1)]['b'])/2)


# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_lin = np.zeros(bus_len)
cost_quad = np.zeros(bus_len)

for i in range(bus_len):
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    cost_lin[i-1] = cost['lin'][k]*base
    cost_quad[i-1] = cost['quad'][k]*base**2

In [229]:
# Define label generation function
def label(idx, line, dist = False):
    #Create a model
    model = pyo.ConcreteModel()

    model.IDX1 = range(gen_len)
    model.IDX2 = range(bus_len)
    model.Pg = pyo.Var(model.IDX2)
    model.Qg = pyo.Var(model.IDX2)
    model.t = pyo.Var(model.IDX2)
    model.v = pyo.Var(model.IDX2)
    
    # Either for Sampling load from a distribution or static load
    if (dist):
        for i in range(bus_len):
            Pd[i] = load_data_sample.loc[idx][i]
            Qd[i] = load_data_sample.loc[idx][i+bus_len]
    else:
        for i in range(bus_len):
            Pd[i] = bus['Pd'][i]/base
            Qd[i] = bus['Qd'][i]/base
     
    # Fix data from generated samples
    for i in model.IDX2:
        model.Pg[i].fix(0) 
        model.Qg[i].fix(0) 

    for i in model.IDX1:
        model.Pg[gen['bus'][i]-1].fix(load_data.loc[idx][1+i])
        model.Qg[gen['bus'][i]-1].fix(load_data.loc[idx][1+i+gen_len])

    #Create slack variables
    model.d1 = pyo.Var(model.IDX2)
    model.d2 = pyo.Var(model.IDX2)

    # declare constraints
    model.c = pyo.ConstraintList()

    for i in range(bus_len):
        model.c.add(expr = model.Pg[i] <= Pmax[i])
        model.c.add(expr = model.Pg[i] >= Pmin[i])
        model.c.add(expr = model.Qg[i] <= Qmax[i])
        model.c.add(expr = model.Qg[i] >= Qmin[i])
        model.c.add(expr = model.v[i] <= Vmax[i])
        model.c.add(expr = model.v[i] >= Vmin[i])    

    #Bus voltage angle initial condition
    model.c.add(model.t[0] == 0)


    #Nodal equations
    for i in range(bus_len):
        model.c.add(expr = sum([model.v[i]*model.v[j]*cmath.polar(Ybus[i,j])[0]*cos(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - model.Pg[i] + Pd[i] == model.d1[i])
        model.c.add(expr = sum([model.v[i]*model.v[j]*cmath.polar(Ybus[i,j])[0]*sin(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - model.Qg[i] + Qd[i] == model.d2[i])


    #Line flow constraints
    for i in range(len(branch)): 
        x = int(branch.loc[i]['fbus'])-1
        y = int(branch.loc[i]['tbus'])-1
        val = (branch.loc[i]['rateA']/base)
        if(val == 0):
            val = 100
    #    With phasor
        Pxy = (model.v[x]**2)*cmath.polar(Ybus[x,y])[0]*cos(cmath.polar(Ybus[x,y])[1]) - model.v[x]*model.v[y]*cmath.polar(Ybus[x,y])[0]*cos(model.t[x]-model.t[y]-cmath.polar(Ybus[x,y])[1])
        Qxy =-(model.v[x]**2)*cmath.polar(Ybus[x,y])[0]*sin(cmath.polar(Ybus[x,y])[1]) - model.v[x]*model.v[y]*cmath.polar(Ybus[x,y])[0]*sin(model.t[x]-model.t[y]-cmath.polar(Ybus[x,y])[1])     

        Pyx = (model.v[y]**2)*cmath.polar(Ybus[x,y])[0]*cos(cmath.polar(Ybus[x,y])[1]) - model.v[x]*model.v[y]*cmath.polar(Ybus[x,y])[0]*cos(model.t[y]-model.t[x]-cmath.polar(Ybus[x,y])[1])
        Qyx =-(model.v[y]**2)*cmath.polar(Ybus[x,y])[0]*sin(cmath.polar(Ybus[x,y])[1]) - model.v[x]*model.v[y]*cmath.polar(Ybus[x,y])[0]*sin(model.t[y]-model.t[x]-cmath.polar(Ybus[x,y])[1])     
        
        #Line disconnection cases; N-1
        if (i == line):
            model.c.add(expr = Pxy == 0)
            model.c.add(expr = Qxy == 0)
            model.c.add(expr = Pyx == 0)
            model.c.add(expr = Qyx == 0)
        else:
            model.c.add(expr = (Pxy**2 + Qxy**2)<= val**2)
            model.c.add(expr = (Pyx**2 + Qyx**2)<= val**2)

    #Declare objective function
    model.cost = pyo.Objective( 
        expr = sum([model.d1[i]**2 for i in range(bus_len)]) + sum([model.d2[i]**2 for i in range(bus_len)]) 
    ) 

    # solve
    solution = SolverFactory('ipopt').solve(model)#.write()
    
    if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
        return model.cost()

    else:
        return 'None'


In [159]:
#Calculate label for different contingencies (Single load case)
arr = [np.array([]) for i in range(len(branch))]
for i in range(len(branch)):
    for j in range(len(load_data)):
        arr[i] = np.append(arr[i],label(j,i))
arr = pd.DataFrame(np.array(arr)).T
# arr <= 1

In [240]:
#Calculate label for different contingencies (load profile following a distribution)
arr_ = [np.array([]) for i in range(len(branch))]
for i in range(len(branch)):
    for j in range(len(load_data)):
        arr_[i] = np.append(arr_[i],label(j,i,True))
arr_ = pd.DataFrame(np.array(arr_)).T
# arr <= 1